## 一、 提出背景

### 1. Internal Covariate Shift

#### 在深层网络训练过程中，由于网络中参数变化而引起内部结点数据分布发生变化的过程：
随着训练的进行，网络中的参数随着梯度下降在不断更新，一方面，底层网络参数发生微弱变化，由于经过多层的线性变换和非线性激活映射使这些微弱变化逐渐被放大；
另一方面，参数变化导致每一层的输入分布发生改变，使得网络一直需要适应分布的变化导致模型训练困难

### 2. ICS造成的问题
- 后层网络需要不断适应输入数据分布的变化，导致网络学习速度降低
- 网络训练过程容易陷入梯度饱和区（梯度为0，参数更新缓慢），减缓网络收敛速度

### 3. 如何减缓ICS
- 白化——PCA白化和ZCA白化
-- 白化过程计算成本太高（每一轮每一层网络都需要白化操作）
-- 白化过程改变了网络每一层的数据分布，从而改变了数据本身的表达能力
- Batch-Normalization

## 二、Batch-Normalization

### 1. 思路
- 由于白化的计算过程复杂，那么可以简化一下，可以尝试对于每个特征进行normalzation，使每个特征均有均值为0，方差为1的分布就OK。
- 另一个问题，白化过程改变每一层网络输入的分布，减弱了数据的表达能力，那么可以考虑加入线性变换操作尽可能恢复数据的表达能力

### 2. 算法
- 深度学习中一般采用mini-batch对网络进行训练，Batch normalization也是基于mini-batch.

#### 2.1 参数定义
- $l: 网络中的层标号$
- $L: 网络中的最后一层或总层数$
- $d_l: 第l层的维度，即神经元结点数$
- $W^{[l]}: 第l层的权重矩阵，W^{[l]}\in \mathbb{R}^{d_l \times d_{l-1}}$
- $b^{[l]}: 第l层的偏置向量，b^{[l]}\in \mathbb{R}^{d_l \times 1}$
- $Z^{[l]}: 第l层的线性计算结果，Z^{[l]}=W^{[l]} \times input+b^{l}$
- $g^{[l]}(.): 第l层的激活函数$
- $A^{[l]}: 第l层的非线性激活结果，A^{[l]}=g^{[l]}(Z{[l]})$
- $M： 样本数量$
- $N： 样本特征数$
- $X： 训练样本集， X=[x_1, x_2, ..., x_M]', X\in \mathbb{R}^{M \times N}$
- $m: batch size 每个batch中的样本数$
- $\chi_i: 第i个mini-batch的训练数据, \chi_i \in \mathbb{R}^{m \times N}$

#### 2.2 算法过程
传入一个batch的样本， 考虑第l层，关注第j个维度，即第j个神经元，则$Z_j^{[l]} \in \mathbb{R}^{m \times 1}$，归一化：
$$\mu_j=\frac{1}{m}\sum_{i=0}^{m}{Z_{ij}^{[l]}}$$
$$\sigma_j^2=\frac{1}{m}\sum_{i=0}^{m}{(Z_{ij}^{[l]}-\mu_j)^2}$$
$$\tilde{{Z_{ij}^{[l]}}}=\frac{Z_{ij}^{[l]}-\mu_j}{\sqrt{\sigma_j^2+\epsilon}}$$
那么，对于神经网络的第l层：
$$Z^{[l]}=W^{[l]}A^{[l-1]}b^{[l]}$$
$$\mu =\frac{1}{m}\sum_{i=0}^{m}{Z_i^{[l]}}$$
$$\sigma^2=\frac{1}{m}\sum_{i=0}^{m}{(Z_i^{[l]}-\mu)^2}$$
$$\tilde{{Z^{[l]}}}=\gamma \frac{Z^{[l]}-\mu}{\sqrt{\sigma^2+\epsilon}}\beta$$
$$A^{[l]}=g^{[l]}(\tilde{Z^{[l]}})$$

## 三、测试阶段如何使用Batch Normalization
在训练阶段，每一层的$\mu$和$\sigma^2$都是基于当前batch的训练样本，而在测试阶段，往往只有一个或者很少的测试样本，此时需要$\mu$和$\sigma^2$的无偏估计：保留每个batch咋每层的$\mu_batch$和$\sigma_batch^2$,使用整个样本的统计量进行估计
$$\mu_{test} = \mathbb{E}(\mu_{batch})$$
$$\sigma_{test}^2=\frac{m}{m-1}\mathbb{E}(\sigma_{batch}^2)$$
$$BN(X_{test})=\gamma \frac{X_{test}-\mu_{test}}{\sqrt{\sigma_{test}^2+\epsilon}}$$

## 四、Batch Normalization的优势
- BN使得网络中每层输入数据的分布相对稳定，加速模型学习速度
- BN使得模型对网络中的参数不那么敏感，简化调参过程，使得网络学习更加稳定
- BN允许使用饱和性激活函数，缓解梯度消失
- BN具有一定的正则化效果